In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
URL = "https://awo-ol.de/meine-awo-karriere/awo-weser-ems/standorte-leistungen"
TIMEOUT = 10

driver = webdriver.Chrome()
driver.maximize_window()
driver.get(URL)

In [3]:
wait = WebDriverWait(driver, TIMEOUT)

In [4]:
while True:
    try:
        button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn.btn-outline-primary.view-more-button"))
            )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
        time.sleep(1)
        button.click()

        time.sleep(3)
    except Exception as e:
        print("Button not found", e)
        break


Button not found Message: 



In [8]:
locations = driver.find_element(By.CSS_SELECTOR, ".row.g-3.standorte-list")
divs_inside = locations.find_elements(By.CSS_SELECTOR, ".filter-item.col-md-6.col-lg-4.standort-append-item")
num_locations = len(divs_inside)

print(num_locations)

107


In [ ]:
def get_details(soup):
    details = {'name': "", 'address': "", 'contact_person': "", 'telephone': "", 'email': ""}

    address_card = soup.find('span', class_='card-title', string=lambda x: x and 'Anschrift' in x)
    # print(address_card)

    if address_card:
        # Get the parent div that contains all address info
        card_body = address_card.find_parent('div', class_='card-body')

        # Find the div after the title that contains the address
        address_div = card_body.find_all('div')[-2]  # Second to last div

        # Extract facility name
        facility_name = address_div.find('span', class_='fw-bold')
        details['name'] = facility_name.text.strip()

        # Extract all span elements
        spans = address_div.find_all('span')
        if len(spans) >= 3:
            add_1 = spans[1].text.strip()
            add_2 = spans[2].text.strip()
            details['address'] = add_1 + '\n' + add_2

    # Get contact person
    contact_heading = soup.find('span', class_='h3 text-primary', string=lambda x: x and 'Ansprechperson' in x)
    if contact_heading:
        card = contact_heading.find_parent('div', class_='card')
        # Get name
        name_span = card.find('span', class_='card-title text-dark')
        details['contact_person'] = name_span.text

    telephone_elem = soup.find('a', href=lambda x: x and x.startswith('tel:'))
    if telephone_elem:
        details['telephone'] = telephone_elem.text

    email_elem = soup.find('a', href=lambda x: x and x.startswith('mailto:'))
    if email_elem:
        details['email'] = email_elem.text

In [ ]:
entries = []
for div in divs_inside:
    try:
        a = div.find_element(By.CSS_SELECTOR, ".btn.btn-link.mt-auto.text-left.stretched-link.position-static")
        facility_url = a.get_attribute("href")
        response = requests.get(facility_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        entries.append(get_details(soup))
    except:
        continue


{'name': 'Altenwohnanlage Marianne-Sternberg-Haus', 'address': 'Anton-Günther-Straße 26\n26441 Jever', 'contact_person': 'Birte-Christine Jansen', 'telephone': '0 44 619 21 60', 'email': 'info@marianne-sternberg-haus.de'}
{'name': 'Altenwohnanlage Rastede', 'address': 'Mühlenstraße 49\n26180 Rastede', 'contact_person': 'Elke Thoben', 'telephone': '0 44 029 28 70', 'email': 'info@altenwohnanlage-rastede.de'}
{'name': 'Altenwohnzentrum Esens', 'address': 'Flack 14\n26427 Esens', 'contact_person': 'Silja Otten', 'telephone': '0 49 719 11 10', 'email': 'info@altenwohnzentrum-esens.de'}
{'name': 'Altenwohnzentrum Haus am Flötenteich', 'address': 'Nadorster Straße 298–302\n26125 Oldenburg', 'contact_person': 'Bianka Vieth', 'telephone': '04 419 31 80', 'email': 'info@haus-am-floetenteich.de'}
{'name': 'Altenwohnzentrum Norden Haus am Kolk', 'address': 'Schulstraße 71\n26506 Norden', 'contact_person': 'Elke Göken', 'telephone': '0 49 3194 50', 'email': 'info@awo-haus-am-kolk.de'}
{'name': 'Al